In [1]:
# deltix TimeBase client
import dxapi
from dxapi import InstrumentMessage, InstrumentType, InstrumentIdentity

# FINOS Perspective
import perspective
from perspective import Table, PerspectiveWidget, Plugin

# OrderBook
from orderbook import Book, to_dict
            
# Other libs
import threading
import asyncio
from datetime import datetime
import time
from datetime import datetime
import ipywidgets as widgets
import logging
from typing import List, Iterable
from sortedcollections import SortedDict, ItemSortedDict
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
tb_info_url = 'dxtick://timebase-headless.timebase5:8011'
schema = {
    'key': str,
    'symbol': str,
    'side': str,
    'size': float,
    'price': float,
    'numberOfOrders': int
}
symbol = 'BTCUSD'
record_type = 'deltix.qsrv.hf.plugins.data.kraken.types.KrakenPackageHeader'
key = 'kraken'
table = Table(schema, limit=1000, index='key')
book = Book(symbol)
booksize = 10
time_widget = widgets.Text(
    disabled=True,
    value=str(datetime.now())
)

In [3]:
def current_milli_time():
    return round(time.time() * 1000)

        
def process_entry_update(symbol, entry: InstrumentMessage) -> None:
    if entry.action == 'DELETE':
        book.remove(entry.side, entry.price)
        if entry.side == 'BID':
            table.update(book.get_bids(size=booksize))
        elif entry.side == 'ASK':
            table.update(book.get_asks(size=booksize))
    elif entry.action == 'UPDATE':
        e = to_dict(symbol, entry)
        book.update(e)
        if entry.level < booksize:
            e['key'] = symbol + entry.side + str(entry.level)
            table.update([e])
    else:
        raise Exception(f'Unknown action type: {entry.action}')

        
def process_entry_new(symbol, entry: InstrumentMessage) -> None:
    e = to_dict(symbol, entry)
    book.update(e)
    if entry.side == 'BID':
        table.update(book.get_bids(size=booksize))
    else:
        table.update(book.get_asks(size=booksize))
            

def process_snapshot(symbol, entries) -> None:
    book.update(*map(lambda e: to_dict(symbol, e), entries))
    table.update(book.get_bids(size=booksize))
    table.update(book.get_asks(size=booksize))
#     logging.info('bids: ' + str(list(map(lambda e: e, book.bids))))
#     logging.info('asks: ' + str(list(map(lambda e: e, book.asks))))
        
        
def initial_book():
    db = dxapi.TickDb_createFromUrl(tb_info_url)
    try:
        db.open(True)
        stream = db.getStream(key)
        options = dxapi.SelectionOptions()
        try:
            cursor = db.select(current_milli_time() - 10000, [stream], options, 
                               [record_type], 
                               [InstrumentIdentity(InstrumentType.FX, symbol)])
            while cursor.next():
                msg = cursor.getMessage()
                if msg.packageType == 'PERIODICAL_SNAPSHOT':
                    return msg
        finally:
            cursor.close()
    finally:
        db.close()

        
async def read_cursor():
    db = dxapi.TickDb_createFromUrl(tb_info_url)
    try:
        db.open(True)
        stream = db.getStream(key)
        options = dxapi.SelectionOptions()
        options.live = True
        try:
            cursor = db.select(current_milli_time(), [stream], options, 
                               [record_type], 
                               [InstrumentIdentity(InstrumentType.FX, symbol)])
            global stop_reading
            initialized = False
            while cursor.next() and not stop_reading and not initialized:
                msg = cursor.getMessage()
                if msg.packageType == 'PERIODICAL_SNAPSHOT' or msg.packageType == 'VENDOR_SNAPSHOT':
                    logging.info('received snapshot')
                    process_snapshot(msg.symbol, msg.entries)
                    initialized = True
                    time_widget.value = str(datetime.fromtimestamp(msg.timestamp / 10 ** 9))
            while cursor.next() and not stop_reading:
                msg = cursor.getMessage()
                if msg.packageType == 'INCREMENTAL_UPDATE':
                    for entry in msg.entries:
                        if entry.typeName.endswith('L2EntryUpdate'):
                            process_entry_update(msg.symbol, entry)
                        elif entry.typeName.endswith('L2EntryNew'):
                            process_entry_new(msg.symbol, entry)
                elif msg.packageType == 'PERIODICAL_SNAPSHOT' or msg.packageType == 'VENDOR_SNAPSHOT':
                    process_snapshot(msg.symbol, msg.entries)
                time_widget.value = str(datetime.fromtimestamp(msg.timestamp / 10 ** 9))
        finally:
            cursor.close()
    finally:
        db.close()

        
def update_table():
    logging.info('Started streaming!')
    loop = asyncio.new_event_loop()
    task = loop.create_task(read_cursor())
    loop.call_later(60, task.cancel)
    
    try:
        loop.run_until_complete(task)
    except asyncio.CancelledError:
        logging.info("Stopped streaming!")
        pass

In [4]:
initial = initial_book()

In [5]:
process_snapshot(initial.symbol, initial.entries)

In [6]:
grid = PerspectiveWidget(
    table, 
    sort=[], 
    row_pivots=['symbol', 'price'], 
    column_pivots=['side'], 
    aggregates={'price': 'avg', 'numberOfOrders': 'sum'},
    columns=['size', 'price', 'numberOfOrders'],
    plugin=Plugin.GRID, 
    dark=True
)
xbar = PerspectiveWidget(
    table, 
    sort=[], 
    columns=['size'],
    row_pivots=['symbol', 'price'], 
    column_pivots=['side'], 
    plugin=Plugin.XBAR,
    dark=True
)
ybar = PerspectiveWidget(
    table, 
    sort=[], 
    columns=['size'],
    row_pivots=['symbol', 'price'], 
    column_pivots=['side'], 
    plugin=Plugin.YBAR,
    dark=True
)
tabs = widgets.Tab()
tabs.children = [grid, xbar, ybar, time_widget]
tabs.set_title(0, 'Grid')
tabs.set_title(1, 'Horizontal Depth Chart')
tabs.set_title(2, 'Vertical Depth Chart')
tabs.set_title(3, 'Timestamp')

In [7]:
tabs

In [8]:
# Execute to start reading messages
stop_reading = False
thread = threading.Thread(target=update_table)
thread.start()

INFO:root:Started streaming!
DEBUG:asyncio:Using selector: EpollSelector
INFO:root:received snapshot


In [10]:
# Execute to stop reading messages
stop_reading = True
thread.join()
table.clear()
grid.clear()
book.clear()